"""

DeltaMC -- Copyright (c) 2025, David Coroian

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

"""

In [1]:
%load_ext Cython

In [ ]:
%%cython
# distutils: language = c++

# Working cell!

import os.path as ospath
import math as mt
import numpy as np
import pandas as pd
import subprocess as subp
from cython.cimports.libcpp.string import string
import cppython_dd_wrapper as deltad_lib
import cython as cppython

# === Paths to I/O files and deps

delta_debugger_project_path = "/path/to/delta-debugger/"
sharpvelvet_path = "path/to/SharpVelvet/"

# velvet report to read initially
initial_report_path = sharpvelvet_path + "out/initial_report.csv"
# path to counter config (one tested, the rest for verifying result)
verif_counters_json_path = delta_debugger_project_path + "dd-configs/counters.json"
# delta debugged counter name from .json file
debugged_counter = "ganak"
# result log file
dd_result_logs = delta_debugger_project_path + "dd-debug/results_probdd_logs_001.txt"

# path to sharpvelvet
velvet_path = sharpvelvet_path + "src/run_fuzzer.py"
# path to delta debugger intermediate cnf output
dd_temp_cnf_output_path = delta_debugger_project_path + "dd-instances/out/dd_output.cnf"
# path to instances.txt - file MUST contain path to DD output
dd_output_instances_path = delta_debugger_project_path + "dd-instances/instances.txt"
# delta debugger reduced CNF output path
reduced_cnf_folder = delta_debugger_project_path + "dd-instances/out/reduced/dd-reduced_"
# delta debugger timelimit per test
dd_test_timeout = "10"
# ====

def IsNan(nr):
  if type(nr) == str:
    return nr == "nan"
  
  return mt.isnan(nr)

def velvet_test_cnf():
  # run velvet
  subp.run(["python",
            velvet_path,
            "-c", 
            verif_counters_json_path,
            "-i", 
            dd_output_instances_path,
            "--timeout",
            dd_test_timeout
            ])

  # read velvet report
  dd_temp_velvet_report_path = sharpvelvet_path + "out/dd-temp_fuzz-results.csv"
  dd_test_report = pd.read_csv(dd_temp_velvet_report_path)

  print("c o [DeltaD] Velvet report:")
  print(dd_test_report)
  print("c o [DeltaD] End Velvet report")

  n_counters = dd_test_report.shape[0]

  majority_threshold = 0
  if n_counters % 2 == 0:
    majority_threshold = n_counters / 2 + 1
  else: 
    majority_threshold = (n_counters + 1) / 2

  counts_array = np.zeros(n_counters, dtype=object)

  # check if it triggered a bug in tested solver
  dd_test_result = False
  debugged_counter_result = np.nan
  for report_instance in dd_test_report.itertuples(index=True):
    counter_name = report_instance.counter
    counts_array[report_instance.Index] = report_instance.count_value
    if counter_name == debugged_counter:
      debugged_counter_result = report_instance.count_value
      if report_instance.timed_out == True or report_instance.error == True:
        dd_test_result = True

  # check wrong count
  # gets unique results and frequency:
  unique_results, frequency_results = np.unique(counts_array, return_counts=True)
  # if debugged counter returned a result
  if IsNan(debugged_counter_result) == False and n_counters > 1:
    # if there is a non-nan majority
    if frequency_results.max() >= majority_threshold and IsNan(unique_results[frequency_results.argmax()]) == False:
      maj_result = unique_results[frequency_results.argmax()]
      # if count differs from majority it is considered wrong
      if maj_result != debugged_counter_result:
        dd_test_result = True
        print(f'c o [DeltaD] - Wrong count detected: {debugged_counter} returns {debugged_counter_result} vs majority {maj_result}')
    else:
      print(f'c o [DeltaD] - No agreement in majority vote')

  return dd_test_result

def delta_debugg(instance, probdd_heuristic = 2):
  # ==== Init delta debugger
  dd_obj = deltad_lib.PyDeltaDebugger()
  dd_obj.init_dd()

  # convert input path to C++ string
  input_path_cpp_string = cppython.declare(string, instance.instance.encode('UTF-8'))
  # read input cnf file 
  dd_obj.read_cnf(input_path_cpp_string)
  # ====

  # ==== Test preprocessed cnf
  # convert output path to C++ string
  output_path_cpp_string = cppython.declare(string, dd_temp_cnf_output_path.encode('UTF-8'))
  dd_obj.print_cnf(output_path_cpp_string)

  # call velvet on preprocessed cnf
  print ("c o [DeltaD] Fuzzing preprocessed CNF")
  preprocessed_cnf_bug = velvet_test_cnf()

  if preprocessed_cnf_bug == False:
    # no bug triggered, return original, preprocessed pair
    print('c o [DeltaD] Finished! Bug not present in preprocessed CNF - reporting (original CNF, preprocessed CNF).')
    return

  print("c o [DeltaD] BUG still present in preprocessed CNF - continuing to ProbDD")
  # ====

  # ==== ProbDD 


  print ("c o [DeltaD] Starting prob-dd")

  log_path_cpp_string = cppython.declare(string, dd_result_logs.encode('UTF-8'))
  counter_name_cpp_string = cppython.declare(string, debugged_counter.encode('UTF-8'))
  dd_obj.init_probdd(probdd_heuristic, log_path_cpp_string, counter_name_cpp_string)

  # prob dd assumes there is at least 1 clause in reduced cnf
  while dd_obj.finished() == False:
    dd_obj.print_test_cnf(output_path_cpp_string)

    tested_cnf_bug = velvet_test_cnf()

    dd_obj.update_prob_dd(tested_cnf_bug)
  
  reduced_cnf_path = reduced_cnf_folder + ospath.basename(instance.instance)
  reduced_cnf_path_cpp_string = cppython.declare(string, reduced_cnf_path.encode('UTF-8'))
  dd_obj.print_final_cnf(reduced_cnf_path_cpp_string)
  
  # ====

# experiments all heuristics
def delta_debugg_all_heuristics(instance):
  for i in [1,2,3]:
    delta_debugg(instance, probdd_heuristic = i)

# ==== DeltaD main: 

# velvet report
cnfs = pd.read_csv(initial_report_path)

print("c o [DeltaD] Initializing DeltaMC...")

for instance in cnfs.itertuples(index=False):
  # Checks whether this instance timed out
  if instance.timed_out:
    print(f'c o [DeltaD] Time out on {instance.instance}!')
    # delta_debugg(instance)
    delta_debugg_all_heuristics(instance)
# ====

